## An unsupervised approach to rank product reviews
### Jianwei Wu ; Bing Xu ; Sheng Li ; 2011

In [1]:
import nltk
import pandas as pd
import numpy as np
import MHR as mhr
import networkx as nx

In [2]:
def zeroFun(x): return 0
dfProducts = pd.read_csv('data/eletronic_sample_counts.csv.gz')
dfProducts = dfProducts[dfProducts.overall==3]
dfProducts['pageRank']=dfProducts.apply(zeroFun,axis=1)
dfProducts.shape

(2075, 20)

In [3]:
def compute_sentence(sentence1, sentence2):
    tags1 = []
    tags2 = []
    
    for token in nltk.pos_tag(nltk.word_tokenize(sentence1.lower()), tagset = "universal"):
        if (token[1] in ['NOUN','VERB','ADJ']):
            tags1.append(token[0])
            
    for token in nltk.pos_tag(nltk.word_tokenize(sentence2.lower()), tagset = "universal"):
        if (token[1] in ['NOUN','VERB','ADJ']):
            tags2.append(token[0])
    
    return (len(set(tags1).intersection(tags2)) > 0)

def compute_sentences_graph(sentences):
    sentences_count = len(sentences)
    matrix = np.zeros((sentences_count,sentences_count))

    for row in range(sentences_count):
        for col in range(sentences_count):
            matrix[row, col] = compute_sentence(sentences[row],sentences[col])

    return matrix

In [ ]:
min_comments=30
min_votes=5

grouped=dfProducts[dfProducts['tot'].astype(int)>min_votes].groupby('asin')
for name, group in grouped:
    
    dffiltro = (dfProducts['asin']==name) & (dfProducts['tot'].astype(int)>min_votes)
    comments_count = dfProducts[dffiltro ]['tot'].values
    
    if ( (len(comments_count)>min_comments) ):
        sentences=[]
        
        for t in dfProducts[dffiltro].T.to_dict().values():
            for s in nltk.sent_tokenize(t['reviewText']):
                sentences.append(s)   
                
        sentences_graph = compute_sentences_graph(sentences)
        nx_graph = nx.from_numpy_matrix(sentences_graph)
        pg_sentences = nx.pagerank_numpy(nx_graph)

        count=0
        for t in dfProducts[dffiltro].T.to_dict().values():
            for s in nltk.sent_tokenize(t['reviewText']):
                dfProducts.loc[dfProducts.reviewerID==t['reviewerID'],'pageRank'] += float(pg_sentences[count])
                count += 1

        break

In [ ]:
dfProducts[dfProducts.pageRank>0]